In [1]:
from random import sample
import pprint
import csv
import random

In [2]:
## Ideally, num_teams * max_team_size = num_devs
## Otherwise, num_teams * (max_team_size - 1) < num_devs <= num_teams * max_team_size, also works
num_teams = 4
num_devs = 22
max_team_size = 6

In [3]:
def stable_marriage(teams_rankings, devs_rankings):
    '''
    Performs stable matching algorithm where each team can propose to up to `num_teams` number of devs
    
    Parameters
    -----------
    teams_rankings: dictionary
        key is team name, value is list of devs where lower index is higher preference
    devs_rankings: dictionary
        key is dev name, value is list of teams where lower index is higher preference
    
    Returns
    -----------
    dev_matching: dictionary
        key is dev, value is the team dev is matched to
    '''
    
    teams = teams_rankings.keys()
    devs = devs_rankings.keys()
    
    matching = [{team: []} for team in teams]
    team_availability = dict(zip(teams, ["free"] * len(teams)))
    team_size = dict(zip(teams, [0] * len(teams)))
    dev_availability = dict(zip(devs, ["free"] * len(devs)))
    team_counter = dict(zip(teams, [0] * len(teams)))
    dev_matching = {}
    
    while not teams_full(team_size):
        if proposals_completed(team_counter):
            break
        
        for team in teams:
            if team_availability[team] != "free" or team_counter[team] >= num_devs:
                continue

            dev_index = team_counter[team]
            dev = teams_rankings[team][dev_index]

            if dev_availability[dev] == "free":
                dev_availability[dev] = "engaged"
                dev_matching[dev] = team
                team_size[team] += 1
                if team_size[team] >= max_team_size:
                    team_availability[team] = "engaged"

            elif dev_availability[dev] == "engaged":
                matched_team = dev_matching[dev]
                proposed_team = team
                dev_rankings = devs_rankings[dev]
                if dev_rankings.index(proposed_team) < dev_rankings.index(matched_team):
                    team_size[matched_team] -= 1
                    if team_availability[matched_team] == "engaged":
                        team_availability[matched_team] = "free"

                    team_size[proposed_team] += 1
                    if team_size[proposed_team] >= max_team_size:
                        team_availability[proposed_team] = "engaged"

                    dev_matching[dev] = proposed_team

            team_counter[team] += 1
    return dev_matching

def teams_full(team_size):
    '''
    Returns true if sum of all team sizes is num_devs
    
    Parameters
    -----------
    team_size: dictionary
        key is team, value is size of team

    Returns
    -----------
    boolean
    '''
    devs_matched = 0
    for _, size in team_size.items():
        devs_matched += size
    return devs_matched == num_devs

def proposals_completed(team_counter):
    '''
    Returns true if all teams have proposed to all devs
    
    Parameters
    -----------
    team_counter: dictionary
        key is team, value is number of devs team has proposed to

    Returns
    -----------
    boolean
    '''
    for _, count in team_counter.items():
        if count < num_devs - 1:
            return False
    return True

def dev_to_team_matching(dev_matching):
    '''
    Converts dev-to-team matching to team-to-dev matching
    
    Parameters
    -----------
    dev_matching: dictionary
        key is dev, value is the team dev is matched to

    Returns
    -----------
    team_matching: dictionary
        key is team, value is list containing devs matched to that team
    '''
    team_matching = {}
    for dev, team in dev_matching.items():
        if team in team_matching.keys():
            team_matching[team].append(dev)
        else:
            team_matching[team] = [dev]
    return team_matching

In [9]:
devs_ranking = {}

with open("dev_ranking.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for i, line in enumerate(reader):
        if i == 0:
            continue
        args = line[0].split(',')
        dev_name = args[0]
        dev_ranking = []
        for i in range(1,5):
            dev_ranking.append(args[i])
        devs_ranking[dev_name] = dev_ranking

In [10]:
pprint.pprint(devs_ranking)

{'Andi Gu': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Andrew Lieu': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Bianca Lee': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Brian DeLeonardis': ['Netskope', 'Pantheon', 'Hackerrank', 'Atlassian'],
 'Carolyn Duan': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Cody Zeng': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Daniel Grimshaw': ['Hackerrank', 'Atlassian', 'Netskope', 'Pantheon'],
 'Erica Kong': ['Hackerrank', 'Atlassian', 'Netskope', 'Pantheon'],
 'Forest Hu': ['Netskope', 'Pantheon', 'Hackerrank', 'Atlassian'],
 'Isabelle Zhou': ['Hackerrank', 'Atlassian', 'Netskope', 'Pantheon'],
 'Ishan Vacchani': ['Netskope', 'Pantheon', 'Hackerrank', 'Atlassian'],
 'Jared Rosner': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Kelvin Jue': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Kush Khanolkar': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Kyle Tse': ['Atlassian'

In [13]:
teams_ranking = {}
ATLASSIAN = -4
HACKERRANK = -3
NETSKOPE = -2
PANTHEON = -1

with open("team_ranking.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    atlassian_ranking = ["dev"] * 22
    hackerrank_ranking = ["dev"] * 22
    netskope_ranking = ["dev"] * 22
    pantheon_ranking = ["dev"] * 22
    
    for i, line in enumerate(reader):
        if i == 0:
            continue
        args = line[0].split(',')
        dev_name = args[0]
        
        atlassian_rank = int(args[ATLASSIAN])
        hackerrank_rank = int(args[HACKERRANK])
        netskope_rank = int(args[NETSKOPE])
        pantheon_rank = int(args[PANTHEON])
        
        atlassian_ranking[atlassian_rank - 1] = dev_name
        hackerrank_ranking[hackerrank_rank - 1] = dev_name
        netskope_ranking[netskope_rank - 1] = dev_name
        pantheon_ranking[pantheon_rank - 1] = dev_name
        
    teams_ranking['Atlassian'] = atlassian_ranking
    teams_ranking['Hackerrank'] = hackerrank_ranking
    teams_ranking['Netskope'] = netskope_ranking
    teams_ranking['Pantheon'] = pantheon_ranking

In [14]:
pprint.pprint(devs_ranking)

{'Andi Gu': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Andrew Lieu': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Bianca Lee': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Brian DeLeonardis': ['Netskope', 'Pantheon', 'Hackerrank', 'Atlassian'],
 'Carolyn Duan': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Cody Zeng': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Daniel Grimshaw': ['Hackerrank', 'Atlassian', 'Netskope', 'Pantheon'],
 'Erica Kong': ['Hackerrank', 'Atlassian', 'Netskope', 'Pantheon'],
 'Forest Hu': ['Netskope', 'Pantheon', 'Hackerrank', 'Atlassian'],
 'Isabelle Zhou': ['Hackerrank', 'Atlassian', 'Netskope', 'Pantheon'],
 'Ishan Vacchani': ['Netskope', 'Pantheon', 'Hackerrank', 'Atlassian'],
 'Jared Rosner': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Kelvin Jue': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Kush Khanolkar': ['Atlassian', 'Netskope', 'Pantheon', 'Hackerrank'],
 'Kyle Tse': ['Atlassian'

In [15]:
pprint.pprint(teams_ranking)

{'Atlassian': ['Nilay Khatore',
               'Trevor Aquino',
               'Erica Kong',
               'Daniel Grimshaw',
               'Isabelle Zhou',
               'Kelvin Jue',
               'Carolyn Duan',
               'Andrew Lieu',
               'Bianca Lee',
               'Andi Gu',
               'Richard Liu',
               'William Zhou',
               'Forest Hu',
               'Lang Tran',
               'Steven Tsay',
               'Ishan Vacchani',
               'Brian DeLeonardis',
               'Kyle Tse',
               'Cody Zeng',
               'Kush Khanolkar',
               'Jared Rosner',
               'Matthew Soh'],
 'Hackerrank': ['Jared Rosner',
                'Matthew Soh',
                'Nilay Khatore',
                'Trevor Aquino',
                'Erica Kong',
                'Daniel Grimshaw',
                'Isabelle Zhou',
                'Kelvin Jue',
                'Carolyn Duan',
                'Andrew Lieu',
          

In [16]:
stable_matching = stable_marriage(teams_ranking, devs_ranking)
team_matching = dev_to_team_matching(stable_matching)

In [17]:
team_matching

{'Atlassian': ['Carolyn Duan',
  'Kelvin Jue',
  'Bianca Lee',
  'Richard Liu',
  'Andi Gu',
  'Andrew Lieu'],
 'Hackerrank': ['Daniel Grimshaw',
  'Trevor Aquino',
  'Nilay Khatore',
  'Isabelle Zhou',
  'Erica Kong',
  'Ishan Vacchani'],
 'Netskope': ['William Zhou',
  'Kush Khanolkar',
  'Cody Zeng',
  'Matthew Soh',
  'Forest Hu',
  'Jared Rosner'],
 'Pantheon': ['Steven Tsay', 'Lang Tran', 'Brian DeLeonardis', 'Kyle Tse']}

In [129]:
with open("output/stable_matching.txt", 'w') as f:
    for key, value in team_matching.items():
        f.write('%s:%s\n' % (key, value))